#### Project Title

- Leslie Cohrt: put their contribution here
- Sarah Auther: put their contribution here
- Shoshana Medved: put their contribution here

#### Introduction

In this study, we aim to understand how people spoke about ChatGPT on Twitter during the first month of its launch. Through this project, we hope to pinpoint any key differences in responses as ChatGPT became more mainstream. By analyzing a data sample of over 200,000 Tweets from November 30th to December 31st, 2022, we will be able to search for key communication trends among users and recurring beliefs about ChatGPT as an emerging platform. 

We are interested in discovering how people view ChatGPT based on data from Twitter
- Through the lens of computer-mediated communication, how has public opinion of ChatGPT evolved as its usage has become more normalized?  
- What generates the most reaction between real people when talking about ChatGPT?

Detailed description of dataset

In [1]:
import pandas as pd

df = pd.read_csv("chatgpt.data.csv")
df.head()

,tweet_id,created_at,like_count,quote_count,reply_count,retweet_count,tweet,country,photo_url,city,country_code
0,1.598010e+18,2022-11-30 18:00:15+00:00,2,0,0,0,ChatGPT: Optimizing Language Models for Dialog...,NaN,NaN,NaN,NaN
1,1.598010e+18,2022-11-30 18:02:06+00:00,12179,889,1130,3252,"Try talking with ChatGPT, our new AI system wh...",NaN,NaN,NaN,NaN
2,1.598010e+18,2022-11-30 18:02:58+00:00,2,0,0,1,ChatGPT: Optimizing Language Models for Dialog...,NaN,https://pbs.twimg.com/media/Fi1J8HbWAAMv_yi.jpg,NaN,NaN
3,1.598020e+18,2022-11-30 18:05:58+00:00,561,8,25,66,"THRILLED to share that ChatGPT, our new model ...",NaN,https://pbs.twimg.com/media/Fi1Km3WUYAAfzHS.jpg,NaN,NaN
4,1.598020e+18,2022-11-30 18:06:01+00:00,1,0,0,0,"As of 2 minutes ago, @OpenAI released their ne...",NaN,NaN,NaN,NaN


In [2]:
# preprocessing

import numpy as np

df1 = df.fillna(value=0)
df1 = df1.loc[df1["like_count"] != 0]

df1.describe()

,tweet_id,like_count,quote_count,reply_count,retweet_count
count,1.201360e+05,120136.000000,120136.000000,120136.000000,120136.000000
mean,1.602453e+18,30.716463,0.716072,2.120688,4.155083
std,2.985152e+15,658.769995,19.037848,34.355344,85.684421
min,1.598010e+18,1.000000,0.000000,0.000000,0.000000
25%,1.600020e+18,1.000000,0.000000,0.000000,0.000000
50%,1.601650e+18,2.000000,0.000000,0.000000,0.000000
75%,1.604640e+18,6.000000,0.000000,1.000000,1.000000
max,1.609340e+18,119321.000000,4598.000000,5184.000000,10593.000000


Our research questions are about shared public opinion and about opinions that generate conversation and response. Tweets with no likes are not indicative of opinions shared by many and do not generate response, so we removed them from the dataset.

In [3]:
!pip install nltk

import nltk

nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

!pip install textblob
from textblob import TextBlob

import re

  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.4/770.4 kB 16.0 MB/s eta 0:00:00a 0:00:01


[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


  Using cached textblob-0.17.1-py2.py3-none-any.whl (636 kB)


In [21]:
# Word tokenization    
text = df1['tweet'].apply(word_tokenize)


In [5]:
print(text)

0         [ChatGPT, :, Optimizing, Language, Models, for...
1         [Try, talking, with, ChatGPT, ,, our, new, AI,...
2         [ChatGPT, :, Optimizing, Language, Models, for...
3         [THRILLED, to, share, that, ChatGPT, ,, our, n...
4         [As, of, 2, minutes, ago, ,, @, OpenAI, releas...
                                ...                        
219287    [One, of, my, new, favorite, thing, to, do, wi...
219288    [Sounds, like, AI, ca, n't, predict, 2023, tre...
219290    [I, asked, #, ChatGPT, to, write, a, #, NYE, J...
219291    [chatgpt, is, being, disassembled, until, it, ...
219292    [2023, predictions, by, #, chatGPT, ., Nothing...
Name: tweet, Length: 120136, dtype: object


In [6]:
import string

In [7]:
exclude = set(string.punctuation)
stop = set(stopwords.words('english'))

print(stop)
print(exclude)

{'mightn', 'on', "couldn't", "weren't", 'that', 'doesn', 'am', 'before', 'didn', 'until', 'ain', 'ma', 'd', 'and', "wasn't", 'her', 'what', 'them', 'are', 'a', 'this', 'as', 'mustn', 'needn', "that'll", 'yourselves', 'any', "don't", 'is', 'aren', 'isn', 'where', 'it', 'will', 'we', 'who', 'its', 'too', 't', 'do', 'into', 'then', 'no', 'which', "she's", 'being', 'under', 'why', 'o', 'once', 'own', 'those', 'all', 'out', 'both', 'yours', "mightn't", 'she', 'each', "wouldn't", 'his', 'more', 'during', 'over', "hadn't", 'ourselves', 'having', "needn't", 'against', 'above', "isn't", 'same', 'does', "you'd", 'can', "doesn't", "you'll", 'he', 'were', 'themselves', 'be', 'of', 'to', 'don', 'nor', 'hasn', 'me', 'when', 'you', 'have', 'wouldn', 're', 'himself', 'below', 'just', 'for', 'or', 'did', 'here', 'very', 'the', 'off', 'itself', 'shan', 'shouldn', "should've", 'weren', 'now', 'their', 'should', 'our', 'through', 'll', 'if', 's', 'again', 'some', 'was', "mustn't", 'in', "you've", "aren't"

In [8]:
filtered_tokens = []
for token in text:
    for word in token:
        if word.lower() not in exclude:
            if word.lower() not in stop:
                filtered_tokens.append(word)

note to self: I think the "too much data" error happened because we were trying to print every word individually from over 200,000 tweets... that's way too many words LOL. I think somewhere she had a way to just make a chart with how many times each word appears? Also more in sentiment analysis (7.1) to look back at later.

#### How have people's opinions of ChatGPT evolved as its usage has become more normalized?
- sentiment analysis
- dependency parsing

#### What generates the most reaction between real people when talking about ChatGPT?
- sentiment analysis

In [60]:
#reducing the data set to the 500 tweets with the most interaction based on a combination of likes, retweets, quotes, and replies.

df_r2 = df1.loc[(df1["like_count"] >299) & (df1["reply_count"] >49) & (df1["retweet_count"] >49) & (df1["quote_count"] >24)]

df_r2.describe()

,tweet_id,like_count,quote_count,reply_count,retweet_count
count,2.500000e+02,250.000000,250.000000,250.000000,250.000000
mean,1.601852e+18,7658.344000,194.260000,315.696000,1052.952000
std,3.320418e+15,12128.445301,368.221374,659.398603,1525.524037
min,1.598010e+18,360.000000,26.000000,50.000000,54.000000
25%,1.599230e+18,2130.750000,43.000000,79.000000,260.000000
50%,1.600480e+18,4072.000000,88.000000,136.500000,520.000000
75%,1.604585e+18,7524.750000,186.750000,243.750000,1116.000000
max,1.609240e+18,119321.000000,4598.000000,5184.000000,10593.000000
